In [ ]:
import purly
import webbrowser
# if you're running on mybinder.org
# localhost does not return 127.0.0.1
from example_utils import localhost

# A Source Of Truth

The core problem that Purly solves is data synchronization between clients - in order to allow Python to control a webpage's Document Object Model (DOM) both Python and the webpage need to share their states with eachother. Purly accomplished this by running a server which acts as a "source of truth" for its clients. This server can be run in its own process via `Machine.run`, but to get started right away we'll use `Machine.daemon` which runs a subprocess.

In [ ]:
purly.state.Machine().daemon(access_log=False)

To start using the Purly model server we'll need to specify the name of the model (``model_name``) our client will view and update. We'll also need to get a URL that we can use to connect to the server. There are two relevant routes.

In [ ]:
model_name = "/example-model"

webpage_url = localhost('http', 8000) + model_name + '/index'
websocket_url = localhost('ws', 8000) + model_name + '/stream'

print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)
print()
print('Websockets use this route to stream layout updates:')
print(websocket_url)

# Making Layouts

In [ ]:
layout = purly.Layout(websocket_url)

In [ ]:
div = layout.html('div')
layout.children.append(div)
div.style.update(height='20px', width='20px', background_color='red')

Now that you've made a layout you need to sync it with the model server with `Layout.sync`

In [ ]:
layout.sync()

# Display Output

Since we're trying to create visual results in the browser we need to show our work. There are a couple ways to do this:


1. **In the notebook**:
    1. Display the `Layout` object in a cell.
    2. Use `purly.display.output` with a websocket route.
2. **In your browser**:
    1. Open a new window and go to `http://my-model-server/<model>`.

In [ ]:
layout

In [ ]:
purly.display.output(websocket_url)

In [ ]:
# You'll have to manually navigate to the webpage if you're at mybinder.org
webbrowser.open(webpage_url)

# Realtime Updates

Because all the displays above are connected to the same model on the same server. They can all be synced in realtime!

In [ ]:
div.style['background_color'] = 'blue'

In [ ]:
layout.sync()

Check out the webpage too. Even it got updated!

# Capturing Events

Purly even allows you to capture events with `purly.HTML.on`.

In [ ]:
@div.on('click')
def toggle(event):
    if div.style['background_color'] == 'blue':
        div.style['background_color'] = 'red'
    else:
        div.style['background_color'] = 'blue'

In [ ]:
layout.serve()

Now go up and click on any of the blue squares we displayed. Thay should all change colors now!